In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
import re

In [3]:
spark = pyspark.sql.SparkSession.builder.appName('WordCount').getOrCreate()

In [4]:
book = spark.read.text('old/Book.txt')
book.show(5)

+--------------------+
|               value|
+--------------------+
|Self-Employment: ...|
|Achieving Financi...|
|       By Frank Kane|
|                    |
|                    |
+--------------------+
only showing top 5 rows



In [5]:
book = book.where(book.value != '')
book.show(5)

+--------------------+
|               value|
+--------------------+
|Self-Employment: ...|
|Achieving Financi...|
|       By Frank Kane|
|Copyright � 2015 ...|
|All rights reserv...|
+--------------------+
only showing top 5 rows



In [6]:
#create the udf
def lenWords(text):
    #normalize words with regex, split on whitespace, convert to lowercase, take the length
    return len(re.compile(r'\W+', re.UNICODE).split(text.lower()))
udfLenWords = pyspark.sql.functions.udf(lenWords)

In [7]:
book = book.withColumn('numWords', udfLenWords('value').cast('integer'))
book.show(5)

+--------------------+--------+
|               value|numWords|
+--------------------+--------+
|Self-Employment: ...|       8|
|Achieving Financi...|      10|
|       By Frank Kane|       3|
|Copyright � 2015 ...|       5|
|All rights reserv...|       5|
+--------------------+--------+
only showing top 5 rows



In [8]:
book.groupBy().sum('numWords').show()

+-------------+
|sum(numWords)|
+-------------+
|        48348|
+-------------+



In [9]:
spark.stop()